<a href="https://colab.research.google.com/github/atotev/ca683-2021-dara/blob/main/Importing_movies.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [99]:
pip install requests

In [100]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [101]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing


In [102]:
apiKey = "defc104635dae32057dd64f95aaf4e0d"


In [103]:
link ='https://drive.google.com/file/d/1rIJjrQh3g6QScqx6J-rN2kC8Ki2weP9v/view?usp=sharing'

In [104]:


downloaded = drive.CreateFile({'id':'1rIJjrQh3g6QScqx6J-rN2kC8Ki2weP9v'}) 
fileMovies = 'movie_titles.csv'

downloaded.GetContentFile(fileMovies)  

movies_netflix = pd.read_csv(fileMovies, 
                           encoding = 'ISO-8859-1', 
                           sep=',',
                           verbose=True,
                           header = None, 
                           names = ['id', 'year', 'title','title_after_comma1', 'title_after_comma2' , 'title_after_comma3']).set_index('id')

print('Shape Movie-Titles:\t{}'.format(movies_netflix.shape))
movies_netflix.sample(5)


Tokenization took: 3.91 ms
Type conversion took: 8.46 ms
Parser memory cleanup took: 0.01 ms
Shape Movie-Titles:	(17770, 5)


,year,title,title_after_comma1,title_after_comma2,title_after_comma3
id,,,,,
9646,1995.0,Sins of the Sisters,NaN,NaN,NaN
16249,1946.0,Humoresque,NaN,NaN,NaN
964,2001.0,The Tick: The Entire Series,NaN,NaN,NaN
17347,2002.0,Read-Along: Monsters,Inc.,NaN,NaN
2871,2003.0,Carrot Top: Rocks Las Vegas,NaN,NaN,NaN


Fix for the 336 movies which are not importorted correcly due to contain comma in the title 

In [105]:
for ind in movies_netflix.index: 
  if not pd.isna(movies_netflix['title_after_comma1'][ind]):
    movies_netflix['title'][ind] = str(movies_netflix['title'][ind]) + str(movies_netflix['title_after_comma1'][ind])
  if not pd.isna(movies_netflix['title_after_comma2'][ind]):
    movies_netflix['title'][ind] = str(movies_netflix['title'][ind]) + str(movies_netflix['title_after_comma2'][ind])
  if not pd.isna(movies_netflix['title_after_comma3'][ind]):
    movies_netflix['title'][ind] = str(movies_netflix['title'][ind]) + str(movies_netflix['title_after_comma3'][ind])

del movies_netflix['title_after_comma1']
del movies_netflix['title_after_comma2']
del movies_netflix['title_after_comma3']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [106]:
movies_netflix.sample(5)

,year,title
id,,
17263,1975.0,Nashville
686,2000.0,Bedazzled
7330,1984.0,Romancing the Stone
13340,2004.0,The Hillside Strangler
1806,1982.0,Gilbert and Sullivan: H.M.S. Pinafore


In [107]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Search for movies in the Movie DB API**
- For each movie found store genre, popularity, votes, genre and media type
- Ignore titles found with category 'People' as it would actors/directores an dnov movies

https://developers.themoviedb.org/3/search/search-movies

In [108]:
def getFirstPartMovieName(movie):
  if movie.find(":") != -1:
    return movie[0:movie.find(":")]
  if movie.find("(") != -1:
    return movie[0:movie.find("(")]
  if movie.find("/") != -1:
    return movie[0:movie.find("/")]
  return movie

def period_is_ok(release_date, year):
    if (len(release_date)>=4 and len(year) >= 4):
      release_date_int = int(float(release_date[0:4]))
      year_int = int(float(year[0:4]))
      difference = (release_date_int - year_int)
      if (difference >= -3 and difference <= 3 ):
        return True
    return False

In [109]:
#add new columns to the data frame
if 'id_moviedb' not in movies_netflix:
  movies_netflix['id_moviedb'] = 0
if 'genre_ids' not in movies_netflix:
  movies_netflix['genre_ids'] = str()
if 'vote_average' not in movies_netflix:
  movies_netflix['vote_average'] = 0
if 'genre_ids' not in movies_netflix:
  movies_netflix['genre_ids'] = 0
if 'media_type' not in movies_netflix:
  movies_netflix['media_type'] = ""
if 'popularity' not in movies_netflix:
  movies_netflix['popularity'] = ""

In [110]:
from google.colab import files
import requests
from time import sleep
count = 0

for ind in movies_netflix.index: 
    movie_title = movies_netflix['title'][ind]
    year = (str(movies_netflix['year'][ind])[0:4])

    movie_title_part = getFirstPartMovieName(movie_title)

    url = "https://api.themoviedb.org/3/search/multi?api_key="+ apiKey +"&language=en-US&query=" + str(movie_title_part)
    response = requests.get(url)
    results = response.json()

    movie = {}

    if (len(results["results"])) >= 1:
      for item in results["results"]:
        if item["media_type"] != "person":
          if "name" in item:            
            if item["name"] == movie_title:
              movie = item
              break
          if "title" in item: 
            if item["title"] == movie_title:
              movie = item
              break
          if "original_title" in item:             
            if item["original_title"] == movie_title:
              movie = item
              break
          if "release_date" in item:                   
            if period_is_ok(item["release_date"], year):
              movie = item
              break
          if "first_air_date" in item:    
            if period_is_ok(item["first_air_date"], year):
              movie = item
              break
             
    elif (len(results["results"])) == 1:
      movie = results["results"][0]
        
    if movie:
      count = count + 1
      movies_netflix['id_moviedb'][ind] = movie["id"]
      if "genre_ids" in movie:
        movies_netflix['genre_ids'][ind] = movie["genre_ids"]
      if "vote_average" in movie:
        movies_netflix['vote_average'][ind] = movie["vote_average"]
      if "media_type" in movie:
        movies_netflix['media_type'][ind] = movie["media_type"]
      if "popularity" in movie:
        movies_netflix['popularity'][ind] = movie["popularity"]

    if count == 10:
      movies_netflix.to_csv('movie_titles_augmented.csv', header=True)
      !cp movie_titles_augmented.csv "gdrive/My Drive/"
      count = 0
      sleep(1.5) #due to the limit of calls of the API

movies_netflix.to_csv('movie_titles_augmented.csv', header=True)
!cp movie_titles_augmented.csv "gdrive/My Drive/"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to b

Total of Movies found in the Movie DB API

In [111]:
movies_netflix[movies_netflix["id_moviedb"] != 0].describe

<bound method NDFrame.describe of          year  ... popularity
id             ...           
1      2003.0  ...      4.927
3      1997.0  ...      0.632
4      1994.0  ...        0.6
5      2004.0  ...       7.08
6      1997.0  ...      5.788
...       ...  ...        ...
17766  2002.0  ...      4.387
17767  2004.0  ...      1.291
17768  2000.0  ...      11.15
17769  2003.0  ...      8.184
17770  2003.0  ...     13.649

[16524 rows x 7 columns]>

**Get List of Genre from API**

In [112]:
url = "https://api.themoviedb.org/3/genre/movie/list?api_key="+ apiKey +"&language=en-US"
response = requests.get(url)
results = response.json()
genres_list = results["genres"]
print(genres_list)

[{'id': 28, 'name': 'Action'}, {'id': 12, 'name': 'Adventure'}, {'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 80, 'name': 'Crime'}, {'id': 99, 'name': 'Documentary'}, {'id': 18, 'name': 'Drama'}, {'id': 10751, 'name': 'Family'}, {'id': 14, 'name': 'Fantasy'}, {'id': 36, 'name': 'History'}, {'id': 27, 'name': 'Horror'}, {'id': 10402, 'name': 'Music'}, {'id': 9648, 'name': 'Mystery'}, {'id': 10749, 'name': 'Romance'}, {'id': 878, 'name': 'Science Fiction'}, {'id': 10770, 'name': 'TV Movie'}, {'id': 53, 'name': 'Thriller'}, {'id': 10752, 'name': 'War'}, {'id': 37, 'name': 'Western'}]


**One-hot enconding**

In [113]:
for ind in movies_netflix.index: 
  for genre in movies_netflix['genre_ids'][ind]:
    genre_found = [x for x in genres_list if x["id"] == int(genre)]
    if genre_found:
      if genre_found[0]["name"] not in movies_netflix:
        movies_netflix[genre_found[0]["name"]] = 0
      movies_netflix[genre_found[0]["name"]][ind] = 1

movies_netflix.to_csv('movie_titles_augmented_onehot_encode.csv', header=True)
!cp movie_titles_augmented_onehot_encode.csv "gdrive/My Drive/"

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


In [114]:
movies_netflix.head(35)

,year,title,id_moviedb,genre_ids,vote_average,media_type,popularity,Adventure,Science Fiction,Documentary,Horror,Action,Animation,Comedy,Thriller,Mystery,Romance,Fantasy,Family,Crime,Drama,Music,History,TV Movie,War,Western
id,,,,,,,,,,,,,,,,,,,,,,,,,,
1,2003.0,Dinosaur Planet,58691,"[12, 878]",3,movie,4.927,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2004.0,Isle of Man TT 2004 Review,0,,0,,,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,1997.0,Character,655068,[99],0,movie,0.632,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,1994.0,Paula Abdul's Get Up & Dance,274766,[],0,movie,0.6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
5,2004.0,The Rise and Fall of ECW,33209,[99],8,movie,7.08,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
6,1997.0,Sick,308867,"[878, 27]",5,movie,5.788,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
7,1992.0,8 Man,666282,"[28, 16, 878]",0,movie,0.6,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
8,2004.0,What the #$*! Do We Know!?,8357,[99],5,movie,8.499,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
9,1991.0,Class of Nuke 'Em High 2,26230,"[35, 27, 878]",4,movie,7.184,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0


**Pre-Processing**

In [115]:
le = preprocessing.LabelEncoder()
# movies_netflix["media_type"] = le.fit_transform(movies_netflix["media_type"].astype(str))
# np.unique(movies_netflix["media_type"])
movies_netflix['popularity'] = pd.to_numeric(movies_netflix['popularity'], errors='coerce')
movies_netflix.dtypes

year               float64
title               object
id_moviedb           int64
genre_ids           object
vote_average         int64
media_type          object
popularity         float64
Adventure            int64
Science Fiction      int64
Documentary          int64
Horror               int64
Action               int64
Animation            int64
Comedy               int64
Thriller             int64
Mystery              int64
Romance              int64
Fantasy              int64
Family               int64
Crime                int64
Drama                int64
Music                int64
History              int64
TV Movie             int64
War                  int64
Western              int64
dtype: object